In [2]:
import pandas as pd

In [35]:
path_deg = '/home/juan/Desktop/juan/bio/mrcv/data/res/04_02_19/DEG-normalized.csv'
path_annotation = '/home/juan/Desktop/juan/bio/data/IWGSC/42/introns.gff3'
path_deg_ann = '/home/juan/Desktop/juan/bio/mrcv/data/res/04_02_19/DEG_annotated.csv'


In [24]:
df_deg = pd.read_csv(path_deg)
print(len(df_deg.index))
df_deg['gene'] = df_deg['gene'].str.split('gene:').str[1]
df_deg.head(2)

3019


,Unnamed: 0,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,C1.counts.txt,C2.counts.txt,C3.counts.txt,C4.counts.txt,T1.counts.txt,T2.counts.txt,T3.counts.txt,T4.counts.txt
0,1,TraesCS2B02G286300,320.013057,-2.445058,0.225052,-10.864391,1.703566e-27,5.369742e-23,582.257421,553.695714,476.353565,549.686360,77.245985,81.333777,93.617622,145.914014
1,2,TraesCS2B02G336800,2294.525203,2.909847,0.268059,10.855257,1.882766e-27,5.369742e-23,532.840739,512.938938,500.171243,609.940442,4976.988116,5270.112921,2148.061490,3805.147731


In [27]:
df_deg = df_deg[['gene','log2FoldChange']]
df_deg.head(2)

,gene,log2FoldChange
0,TraesCS2B02G286300,-2.445058
1,TraesCS2B02G336800,2.909847


In [26]:
gene_list = df_deg.gene.tolist()

In [18]:
df_ann = pd.read_csv(path_annotation,sep="\t", header=None)
df_ann.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
print(len(df_ann.index))
df_ann.head(2)

2573233


,seqname,source,feature,start,end,score,strand,frame,attribute
0,1A,International Wheat Genome Sequencing Consortium,chromosome,1,594102056,.,.,.,"ID=chromosome:1A;Alias=chr1A,LS992080.1"
1,1A,Ensembl_Plants,ncRNA_gene,200,1683,.,+,.,ID=gene:ENSRNA050013875;Name=LSU_rRNA_eukarya;...


In [19]:
df_ann = df_ann[df_ann.feature == 'gene']
print(len(df_ann.index))
df_ann.head(2)

107891


,seqname,source,feature,start,end,score,strand,frame,attribute
34,1A,IWGSC,gene,40098,70338,.,-,.,ID=gene:TraesCS1A02G000100;biotype=protein_cod...
67,1A,IWGSC,gene,70239,89245,.,+,.,ID=gene:TraesCS1A02G000200;biotype=protein_cod...


In [20]:
def filter(x):
    gene_name = x.attribute.split('gene:')[1]
    gene_name = gene_name.split(';')[0]
    return gene_name in gene_list
df_ann = df_ann[df_ann.apply(lambda x: filter(x), axis=1)]
print(len(df_ann.index))
df_ann.head(2)

3018


,seqname,source,feature,start,end,score,strand,frame,attribute
863,1A,IWGSC,gene,1188844,1190864,.,+,.,ID=gene:TraesCS1A02G002200;biotype=protein_cod...
1912,1A,IWGSC,gene,3500137,3503929,.,-,.,ID=gene:TraesCS1A02G006300;biotype=protein_cod...


In [22]:
df_ann['gene'] = df_ann.attribute.str.split('gene:').str[1]
df_ann['gene'] = df_ann.gene.str.split(';').str[0]
df_ann.head(2)

,seqname,source,feature,start,end,score,strand,frame,attribute,gene
863,1A,IWGSC,gene,1188844,1190864,.,+,.,ID=gene:TraesCS1A02G002200;biotype=protein_cod...,TraesCS1A02G002200
1912,1A,IWGSC,gene,3500137,3503929,.,-,.,ID=gene:TraesCS1A02G006300;biotype=protein_cod...,TraesCS1A02G006300


In [23]:
df_ann = df_ann[['gene','seqname','start','end']]
df_ann.head(2)

,gene,seqname,start,end
863,TraesCS1A02G002200,1A,1188844,1190864
1912,TraesCS1A02G006300,1A,3500137,3503929


In [36]:
df = pd.merge(df_deg, df_ann)
df.head(2)

,gene,log2FoldChange,seqname,start,end
0,TraesCS2B02G286300,-2.445058,2B,394317533,394318758
1,TraesCS2B02G336800,2.909847,2B,481048634,481054363


In [37]:
df.to_csv(path_deg_ann, sep='\t', index=None)